In [ ]:
### Uncomment below lines if you are just trying to run this notebook and have not done a full installation of this project
# !pip install openai==1.41.0 faiss-cpu==1.7.4 wandb==0.16.3 PyYAML==6.0.2 tiktoken==0.3.3 python-dotenv==1.0.1
# !pip install llama-index-core==0.10.13 llama-index-readers-file==0.1.6 llama-index-embeddings-openai==0.1.6 llama-index-vector-stores-faiss==0.1.2 llama-index-callbacks-wandb==0.1.2

In [ ]:
from pathlib import Path
from dotenv import load_dotenv

# Determine the root directory and the path to the .env file
root_dir = Path.cwd().parent.parent
dotenv_path = root_dir / ".env"

# Load the .env file
load_dotenv(dotenv_path)

In [ ]:
import tiktoken
from chat_with_repo.filesystem import Filesystem
from chat_with_repo.clone import clone_repo_and_checkout_commit
from chat_with_repo.chat.open_ai import chat_completion_request

In [ ]:
repo = "pixijs/pixijs"
fs = Filesystem(path_storage="./wiki/", path_benchmark="./wiki/benchmark/")
repo_info = {
    "commit": "b631cc57fa3b039211b0401e0317b5bdddb51167",
    "branch": "dev",
    "url": "https://github.com/pixijs/pixijs.git",
}
# check if we already have the repo
if not (fs.path_apps / repo).exists():
    clone_repo_and_checkout_commit(fs, repo, repo_info)

In [ ]:
def find_render_methods(file_paths):
    relevant_files = []
    for file_path in file_paths:
        # double check this is a file not a directory
        if not file_path.is_file():
            continue
        # double check not a test file
        if (
            "test" in file_path.name
            or "/test" in str(file_path)
            or "/tests/" in str(file_path)
        ):
            continue
        with open(file_path, "r") as file:
            source_code = file.read()
        if ".render(" in source_code or " render(" in source_code:
            relevant_files.append(file_path)
    return relevant_files


repo_path = fs.path_apps / repo
file_paths = list(repo_path.glob("**/*.ts"))

files_with_render_functions = find_render_methods(file_paths)

# Print the found render methods
# for f in files_with_render_functions:
# print(f"Found file with a render method or function: {str(f).split('/pixijs/pixijs/')[1]}")
print(len(files_with_render_functions))

--> 33 files with mentions of ".render(" or " render("

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


documents_str = ""
for doc in files_with_render_functions:
    # load
    with open(doc, "r") as file:
        documents_str += "'''" + str(doc).split("/pixijs/pixijs/")[1] + "''':\n"
        documents_str += file.read() + "\n----------------\n"

# calculate metrics for this: number of tokens, cost of completion
num_tokens = num_tokens_from_string(documents_str, "cl100k_base")
cost_per_1k_tokens = 0.01  # https://openai.com/pricing
cost = cost_per_1k_tokens * (num_tokens / 1000)
print(f"input cost ~ USD ${round(cost, 2)} for {num_tokens} tokens")

--> Cost of processing document tokens = $0.86 USD

In [ ]:
# get a chat completion
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that explains how the code works in a given repository.",
    },
    {
        "role": "user",
        "content": "Can you explain how the render methods work in this project? Is there one specific `render` method always called? What are the different render methods?",
    },
    {"role": "function", "name": "query_repository_files", "content": documents_str},
]

print(chat_completion_request(messages))

#### response:

The code provided consists of multiple parts, originating from different files across a repository. The files implement various features related to rendering graphical objects, particle systems, and sprites in a PixiJS (a 2D rendering library for the web).

Here is a summary of each file's content:

1. `packages/mesh/src/Mesh.ts`:
   - Defines a `Mesh` class that can render WebGL visuals such as geometry, shader, and state.
   - Includes methods to configure and render a mesh object.

2. `packages/canvas-sprite/src/Sprite.ts`:
   - Extends a `Sprite` prototype to allow rendering of sprites on a 2D canvas.
   - Contains an overridden `_renderCanvas` method specific for canvas rendering.

3. `packages/canvas-sprite/src/CanvasSpriteRenderer.ts`:
   - Defines a `CanvasSpriteRenderer` class responsible for rendering sprites on a canvas using 2D context methods.
   - It is designed for the canvas renderer plugin.

4. `packages/core/src/Renderer.ts`:
   - Implements the core `Renderer` class of PixiJS, including methods and properties for WebGL rendering tasks.
   - This class represents the system that manages the WebGL context and performs rendering operations.

5. `packages/core/src/IRenderer.ts`:
   - Contains the interfaces for PixiJS renderer types, including `IRenderer`, `IRendererOptions`, `IRendererRenderOptions`, and `IRenderableObject`.
   - These interfaces define the required properties and methods for renderers in PixiJS.

6. `packages/core/src/framebuffer/Framebuffer.ts`:
   - Defines a `Framebuffer` class which encapsulates the WebGL framebuffer and associated state.
   - The framebuffer is a key concept in graphics rendering pipelines, allowing off-screen rendering and other advanced effects.

7. `packages/core/src/renderTexture/RenderTexture.ts`:
   - Provides a `RenderTexture` class for PixiJS, representing a texture onto which you can render display objects.
   - RenderTextures are essential for creating dynamic textures at runtime.

8. `packages/core/src/renderTexture/BaseRenderTexture.ts`:
   - Introduces a `BaseRenderTexture` class, a base class for `RenderTexture`, managing the underlying WebGL texture and associated properties.

9. `packages/core/src/renderTexture/GenerateTextureSystem.ts`:
   - Implements a `GenerateTextureSystem` that manages the generation of textures from display objects.
   - This system is used for caching complex display objects as textures for performance optimization.

10. `packages/core/src/render/ObjectRendererSystem.ts`:
    - Provides an `ObjectRendererSystem` responsible for rendering objects with PixiJS using different renderers such as WebGL or canvas.

11. `packages/core/src/batch/ObjectRenderer.ts`:
    - A base class for custom renderers that handle batching of objects.
    - Batching is a performance optimization technique for rendering many objects with a single draw call.

12. `packages/core/src/batch/BatchRenderer.ts`:
    - A `BatchRenderer` class that manages the rendering of sprites and graphics in batches to improve performance.
    - It uses the concept of batches to group rendering operations efficiently.

13. `packages/app/src/ResizePlugin.ts`: 
    - A `ResizePlugin` that provides functionality for automatically resizing a PixiJS application based on the window or specified HTML element size.
    
14. `packages/app/src/Application.ts`:
    - Defines the `Application` class, which simplifies the setup and management of a PixiJS application by handling the renderer, ticker, and root container creation.

15. `packages/canvas-mesh/src/MeshMaterial.ts`:
    - Extends the `MeshMaterial` to add a method for rendering meshes using the canvas renderer (`_renderCanvas`).

16. `packages/canvas-mesh/src/CanvasMeshRenderer.ts`:
    - Introduces a `CanvasMeshRenderer` class responsible for rendering mesh objects on a 2D canvas.

17. `packages/canvas-graphics/src/CanvasGraphicsRenderer.ts`:
    - A `CanvasGraphicsRenderer` that handles the rendering of graphics objects to a 2D canvas context, managing line styles, fills, and other graphics drawing operations.

18. `packages/canvas-graphics/src/Graphics.ts`:
    - Extends the `Graphics` class to override methods that execute rendering of shapes on a 2D canvas.

19. `packages/particle-container/src/ParticleContainer.ts`:
    - A `ParticleContainer` class that offers a high-performance container for managing and rendering large numbers of similar display objects such as particles.

20. `packages/particle-container/src/ParticleRenderer.ts`:
    - A renderer class (`ParticleRenderer`) specialized for performing optimized rendering of particle systems.

21. `packages/mixin-cache-as-bitmap/src/index.ts`:
    - Mixin functionality to cache display objects as bitmaps for performance optimization.

22. `packages/display/src/DisplayObject.ts`:
    - The base class (`DisplayObject`) for all objects in PixiJS that are displayable, defining transformations, visibility, interaction, and rendering behavior.

23. `packages/display/src/Container.ts`:
    - A `Container` class that extends `DisplayObject` and allows containing and managing other display objects, handling their rendering and transformations in a hierarchical structure.

24. `packages/graphics/src/Graphics.ts`:
    - A `Graphics` class that allows drawing shapes such as lines, circles, rectangles, and also coloring them. It's akin to a drawing canvas with an API to draw various graphic primitives.

Each class and file is part of the larger PixiJS framework structure, which facilitates the rendering of 2D graphics in web applications via WebGL or HTML canvas.

In [ ]:
documents_str = ""
for file_path in (fs.path_apps / repo / "packages/core/src/renderTexture").glob(
    "**/*.ts"
):
    with open(file_path, "r") as file:
        documents_str += "'''" + str(file_path).split("/pixijs/pixijs/")[1] + "''':\n"
        documents_str += file.read() + "\n----------------\n"

# calculate metrics for this: number of tokens, cost of completion
num_tokens = num_tokens_from_string(documents_str, "cl100k_base")
cost_per_1k_tokens = 0.01  # https://openai.com/pricing
cost = cost_per_1k_tokens * (num_tokens / 1000)
print(f"input cost ~ USD ${round(cost, 2)} for {num_tokens} tokens")

--> Cost of processing document tokens = $0.07 USD

In [ ]:
# get a chat completion
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that explains how the code works in a given repository.",
    },
    {
        "role": "user",
        "content": "Which render function is the last to be called in our set of files here?",
    },
    {"role": "function", "name": "query_repository_files", "content": documents_str},
]

print(chat_completion_request(messages))

#### Response

The code provided consists of TypeScript classes, each pertaining to different aspects of render texture management in a rendering pipeline, likely for a WebGL context like PixiJS. The classes involved are:

1. `RenderTexturePool` - Manages a pool of `RenderTexture` objects for efficient memory use. Render textures can be created, retrieved, and returned to the pool.
2. `RenderTexture` - Represents a texture that can be rendered to. It is essentially a special type of texture within the rendering system.
3. `BaseRenderTexture` - The base class for a render texture that any PixiJS display object can render to.
4. `RenderTextureSystem` - A renderer system plugin managing render textures, responsible for binding and clearing render textures.
5. `GenerateTextureSystem` - A system for generating textures from the renderer.

To understand which render function is the last to be called, we need to analyze the functionality provided by each class. However, none of the classes directly include a method named `render`. Instead, they provide functionality for creating, managing, and binding render textures, but not explicitly rendering them.

From this code alone, one could deduce that the actual rendering to a render texture would be done through the renderer's render method, not through these classes directly. For example, in `GenerateTextureSystem.generateTexture`, we see a call to `this.renderer.render(displayObject, {...})`. This is likely where the drawing operation takes place, and it uses a `renderTexture` as the target for rendering.

Considering all the classes involved, `GenerateTextureSystem.generateTexture` seems to be the last point in this code where something is rendered to a texture, as it involves drawing a display object onto a render texture. However, since the actual rendering function (`this.renderer.render`) is not included in the provided code, I cannot definitively say which is the last render function to be called without more context or access to the `renderer.render` implementation within the renderer class.

In [ ]:
documents_str = ""
for file_path in [
    (fs.path_apps / repo / "packages/core/src/Renderer.ts"),
    (fs.path_apps / repo / "packages/canvas-sprite/src/CanvasSpriteRenderer.ts"),
    (fs.path_apps / repo / "packages/core/src/render/ObjectRendererSystem.ts"),
]:
    with open(file_path, "r") as file:
        documents_str += "'''" + str(file_path).split("/pixijs/pixijs/")[1] + "''':\n"
        documents_str += file.read() + "\n----------------\n"

# calculate metrics for this: number of tokens, cost of completion
num_tokens = num_tokens_from_string(documents_str, "cl100k_base")
cost_per_1k_tokens = 0.01  # https://openai.com/pricing
cost = cost_per_1k_tokens * (num_tokens / 1000)
print(f"input cost ~ USD ${round(cost, 2)} for {num_tokens} tokens")

--> Cost of processing document tokens = $0.07 USD

In [ ]:
# get a chat completion
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that explains how the code works in a given repository.",
    },
    {
        "role": "user",
        "content": "Does the ObjectRenderer render() function get called for every single entity that is rendered by Pixi?",
    },
    {"role": "function", "name": "query_repository_files", "content": documents_str},
]

print(chat_completion_request(messages))

#### Response

The code snippets you've provided are from different parts of the PixiJS library. PixiJS is a 2D rendering library that allows for WebGL-based rendering with a Canvas fallback. It provides a system for rendering sprites, graphics, text, and other visual elements in a scene graph. Let's go through the relevant parts of the code you've given regarding your question about the `ObjectRenderer`'s `render()` function.

In the `Renderer` class from `packages/core/src/Renderer.ts`, we find the instance method `render()` defined as:

```typescript
render(displayObject: IRenderableObject, options?: IRendererRenderOptions): void
{
    this.objectRenderer.render(displayObject, options);
}
```

This method is used to render a `displayObject`, which is any object that can be rendered within PixiJS's scene graph. The actual work for rendering is delegated to the `this.objectRenderer`, an instance of `ObjectRendererSystem`.

The `ObjectRendererSystem` is defined in `packages/core/src/render/ObjectRendererSystem.ts`, with its `render()` method having the following signature:

```typescript
render(displayObject: IRenderableObject, options?: IRendererRenderOptions): void
{
    // Implementation details...
}
```

Here is where the actual work of rendering a `displayObject` happens. There's a checking process for whether the context is lost, a transformation update for the display object, and a series of steps to bind the `renderTexture`, clear if necessary, and invoke the `render()` method of the `displayObject`.

This `render()` function does not automatically get called for every single entity in the scene graph. Instead, it is the method that you, as a user of PixiJS, would call when you want to render a particular display object, potentially into a specific `renderTexture`. More commonly, PixiJS maintains a render loop where this method is called for the stage (the root of the scene graph), which then recursively renders all the children.

In the `CanvasSpriteRenderer` outlined in `packages/canvas-sprite/src/CanvasSpriteRenderer.ts`, we see an example of how a specific type of object—`Sprite` in this case—is rendered in the context of the canvas renderer plugin. Here, `render()` is more of an override that handles the unique requirements for rendering a sprite on the canvas.

In summary, `ObjectRendererSystem.render()` is the main point of rendering a display object within PixiJS, and it is typically called as part of the rendering process initiated by the user or within the library's rendering loop. Each display object's own `render()` method is called during the scene graph traversal, allowing for a hierarchical and efficient rendering process.

In [ ]:
documents_str = ""
for file_path in [
    (fs.path_apps / repo / "packages/core/src/Renderer.ts"),
    (fs.path_apps / repo / "packages/core/src/framebuffer/Framebuffer.ts"),
    (fs.path_apps / repo / "packages/particle-container/src/ParticleRenderer.ts"),
    (fs.path_apps / repo / "packages/graphics/src/Graphics.ts"),
    (fs.path_apps / repo / "packages/mesh/src/Mesh.ts"),
]:
    with open(file_path, "r") as file:
        documents_str += "'''" + str(file_path).split("/pixijs/pixijs/")[1] + "''':\n"
        documents_str += file.read() + "\n----------------\n"

# calculate metrics for this: number of tokens, cost of completion
num_tokens = num_tokens_from_string(documents_str, "cl100k_base")
cost_per_1k_tokens = 0.01  # https://openai.com/pricing
cost = cost_per_1k_tokens * (num_tokens / 1000)
print(f"input cost ~ USD ${round(cost, 2)} for {num_tokens} tokens")

--> input cost ~ USD $0.22 for 21752 tokens

In [ ]:
user_msg = """\
Is there a single render function that is shared amongst these code files? 
In other words, are particles rendered seperately from graphics which are rendered seperately from objects?
What method actually generates the bitmap for a canvas (/webgl) frame?\
"""
# get a chat completion
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that explains how the code works in the [PixiJS](https://github.com/pixijs/pixijs) repository.",
    },
    {
        "role": "user",
        "content": user_msg,
    },
    {"role": "function", "name": "query_repository_files", "content": documents_str},
]

print(chat_completion_request(messages))

#### Response

PixiJS does not have a single shared render function across the different rendering systems (such as particles, graphics, or mesh objects). Instead, PixiJS utilizes a flexible rendering architecture where different systems handle their own rendering based on the type of object and the rendering backend (WebGL/canvas).

Let's explore each part according to your question:

1. Particles: Particles are handled by the ParticleRenderer. The particles are batched and rendered according to their own logic, found in the `ParticleRenderer` class.

2. Graphics: Graphics objects (such as lines, shapes, fills) are rendered by their own system too. The meat of the rendering logic starts in the `Graphics.render` method, which delegates rendering to either a batch renderer or using a direct render method based on certain conditions.

3. Mesh/Objects: Mesh objects have their rendering logic encapsulated within their class. The `Mesh` class defines its `_render` method, which checks whether the mesh can be batch rendered or needs to be rendered directly and handles the binding and drawing of geometry.

4. Generation of bitmap for canvas/WebGL frame: The actual rendering to a canvas or WebGL frame is handled through various renderer and system classes that manage the WebGL state, bind buffers, and execute draw calls. For example, the `Renderer` class is the main driver, it orchestrates the rendering process by using systems (like `BatchSystem`, `TextureSystem`, `StateSystem`, etc.), plugins, and rendering contexts (WebGL).

The above files show that PixiJS has a modular and extensible rendering system. Each type of graphics entity is responsible for its own rendering logic, while PixiJS provides the infrastructure to manage the rendering process, state, and context. Each renderer has its own set of rules for how objects are batched or drawn directly to the framebuffer that eventually generates the image you see on a canvas or WebGL context.

--- 

--> ... *manual verification*

#### List of PixiJS files with methods that render to screen
##### `packages/core/src/render/ObjectRendererSystem.ts`
- `ObjectRendererSystem.prototype.render`
- Main method for rendering objects onto the screen (objects are implemented with `IRenderableObject`)
- Objects must be implemented with their own `render()` method (or in some cases, a `_render()` method, e.g., when inheriting from `Container`)
    - But it seems like something like `Sprite` just has its texture displayed, which is drawn to WebGL with `updateTexture()`, but it is not clear how PixiJS tells WebGL where to draw this texture (using its `localTransform`)

##### `packages/core/src/batch/BatchRenderer.ts`
- `BatchRenderer.prototype.flush` (?)
- Inherits from `Renderer` (which calls `ObjectRendererSystem.prototype.render`) and does WebGL draw calls in batches

##### `packages/core/src/geometry/GeometrySystem.ts`
- `GeometrySystem.prototype.draw`
- Called by `packages/mesh/src/Mesh.ts` to draw meshes 
    - Has `Mesh.prototype._render` method
- Called by `packages/graphics/src/Graphics.ts`to draw primitive shapes
    - Has `Graphics.prototype._render` method

##### `packages/particle-container/src/ParticleRenderer.ts`
- `ParticleRenderer.prototype.render`
- Renders the `ParticleContainer` object.

##### `packages/sprite-tiling/src/TilingSpriteRenderer.ts`
- `TilingSpriteRenderer.prototype.render`
- Default render method used for `TilingSprite` objects

##### `packages/canvas-renderer/src/CanvasObjectRendererSystem.ts`
- 

##### `packages/canvas-sprite/src/CanvasSpriteRenderer.ts`
-

##### `packages/canvas-graphics/src/CanvasGraphicsRenderer.ts`
-

##### `packages/canvas-mesh/src/CanvasMeshRenderer.ts`
-

### TODO
- How do rendering calls work with `requestAnimationFrame` - are we updating on every `requestAnimationFrame()` call?
- It seems like the actual WebGL draw calls are all handled in the `BatchRenderer` class(es). Confirm this?

--> Moved out of Jupyter Notebook and made "PixiJS Explainer" GPT Assistant by uploading the source code in a zip file.

#### Load up the webgl matches

In [ ]:
# load the file ./storage/webgl_usages_in_pixijs.csv
import pandas as pd

path_parent_storage = "./storage/"

df = pd.read_csv(path_parent_storage + "webgl_usages_in_pixijs.csv")

# print all the relative file paths (start from pixijs/pixijs/.)

for x in (
    df["file"]
    .str.split("/mnt/data/source_code_reextracted/pixijs/")
    .apply(lambda x: x[1] if len(x) > 1 else x[0])
):
    print(f"'{x}'")

#### Run code to construct call graph of WebGL 

--> See `understanding_pixi/buildCallGraph.ts`

#### Use PixiJS Explainer GPT to find the files that utilize WebGL draw calls

--> output file saved to `./storage/pixijs_webgl_paths.json`

In [ ]:
import json
import os

# Load the data from the JSON file
with open("./storage/pixijs_webgl_paths.json", "r") as file:
    data = json.load(file)

# Prepend the paths with the correct prefix
data = {call: [f"pixijs/{p}" for p in paths] for call, paths in data.items()}

# Iterate over each call and its paths
for call, paths in data.items():
    print(f"Call: {call}")
    for path in paths:
        print(f"Path: {path}")
        # Get the absolute path
        abs_path = os.path.abspath(path)

        # Open the file
        with open(abs_path, "r") as file:
            # Read the lines
            lines = file.readlines()

            # Find the line that has the call
            for line_num, line in enumerate(lines, start=1):
                if call in line:
                    print(f"Line {line_num}: {line.strip()}")
                    break
        print("----------------")

```
Call: drawArrays

Path: pixijs/pixijs/packages/core/src/geometry/GeometrySystem.ts
Line 131: gl.drawArraysInstanced = (a, b, c, d): void =>

Path: pixijs/pixijs/packages/graphics/src/utils/buildLine.ts
Line 589: * Builds a line to draw using the gl.drawArrays(gl.LINES) method

----------------

Call: bindBuffer

Path: pixijs/pixijs/packages/core/src/shader/ShaderSystem.ts
Line 207: this.renderer.buffer.bindBufferBase(group.buffer, glProgram.uniformBufferBindings[name]);

Path: pixijs/pixijs/packages/core/src/geometry/BufferSystem.ts
Line 86: gl.bindBuffer(buffer.type, glBuffer.buffer);

----------------

Call: texImage2D

Path: pixijs/pixijs/packages/core/src/textures/TextureSystem.ts
Line 112: gl.texImage2D(gl.TEXTURE_2D, 0, gl.RGBA, 1, 1, 0, gl.RGBA, gl.UNSIGNED_BYTE, new Uint8Array(4));

Path: pixijs/pixijs/packages/core/src/textures/resources/BaseImageResource.ts
Line 23: * If set to `true`, will force `texImage2D` over `texSubImage2D` for uploading.

Path: pixijs/pixijs/packages/core/src/textures/resources/ImageResource.ts
Line 242: // bitmap stores unpack premultiply flag, we dont have to notify texImage2D about it

Path: pixijs/pixijs/packages/core/src/textures/resources/CubeResource.ts
Line 159: renderer.gl.texImage2D(side.target, 0,

Path: pixijs/pixijs/packages/core/src/textures/resources/BufferResource.ts
Line 91: gl.texImage2D(

Path: pixijs/pixijs/packages/utils/src/browser/detectVideoAlphaMode.ts
Line 66: gl.texImage2D(gl.TEXTURE_2D, 0, gl.RGBA, gl.RGBA, gl.UNSIGNED_BYTE, video);
```

#### Start writing bug injection code

Doing Just-in-Time (JiT) bug injection by extending PixiSampler: https://github.com/asgaardlab/canvas-visual-bugs-testbed


----